In [2]:
!pip install transformers datasets seqeval tokenizers accelerate torch -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 15.6 MB/s eta 0:00:00


## Loading Data

In [3]:
import datasets
import numpy as np
from transformers import (BertTokenizerFast,
                          DataCollatorForTokenClassification,
                          AutoModelForTokenClassification)

In [4]:
data = datasets.load_dataset("conll2003", trust_remote_code=True)
data

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [5]:
data.shape

{'train': (14041, 5), 'validation': (3250, 5), 'test': (3453, 5)}

ner_tags: a list of classification labels (int)
- {'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6, 'B-MISC': 7, 'I-MISC': 8}

In [6]:
data['train'][0]

{'id': '0',
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}

In [7]:
data['train'].description

'The shared task of CoNLL-2003 concerns language-independent named entity recognition. We will concentrate on\nfour types of named entities: persons, locations, organizations and names of miscellaneous entities that do\nnot belong to the previous three groups.\n\nThe CoNLL-2003 shared task data files contain four columns separated by a single space. Each word has been put on\na separate line and there is an empty line after each sentence. The first item on each line is a word, the second\na part-of-speech (POS) tag, the third a syntactic chunk tag and the fourth the named entity tag. The chunk tags\nand the named entity tags have the format I-TYPE which means that the word is inside a phrase of type TYPE. Only\nif two phrases of the same type immediately follow each other, the first word of the second phrase will have tag\nB-TYPE to show that it starts a new phrase. A word with tag O is not part of a phrase. Note the dataset uses IOB2\ntagging scheme, whereas the original dataset uses 

## Tokenzier

In [8]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [9]:
text = data['train']['tokens'][0]
tokenized_text = tokenizer(text, is_split_into_words=True)
tokenized_text

{'input_ids': [101, 7327, 19164, 2446, 2655, 2000, 17757, 2329, 12559, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [10]:
tokens = tokenizer.convert_ids_to_tokens(tokenized_text['input_ids'])
tokens

['[CLS]',
 'eu',
 'rejects',
 'german',
 'call',
 'to',
 'boycott',
 'british',
 'lamb',
 '.',
 '[SEP]']

In [11]:
word_ids = tokenized_text.word_ids()
word_ids

[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, None]

In [12]:
data['train']['ner_tags'][0]

[3, 0, 7, 0, 0, 0, 7, 0, 0]

In [13]:
len(data['train']['ner_tags'][0]), len(tokenized_text['input_ids'])

(9, 11)

Transformers in general follow subword tokenization. This means that the already split words will be split further into smaller parts. The tokenizer also adds special tokens to the input, which are required for the model to work properly.

However, this results in a mismatch between the tokens generated by the tokenizer and the actual labels. To fix this, the tokens and labels need to be aligned.

In [14]:
#align the labels with the tokenized text and in place of special tokens, add -100
def tokenize_and_align_labels(input, label_all_tokens=True):
    tokenized_input = tokenizer(input['tokens'], is_split_into_words=True)
    labels = []
    for i, label in enumerate(input['ner_tags']):
        word_ids = tokenized_input.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_input["labels"] = labels
    return tokenized_input

In [15]:
tokenize_and_align_labels(data['train'][0:1])

{'input_ids': [[101, 7327, 19164, 2446, 2655, 2000, 17757, 2329, 12559, 1012, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, -100]]}

In [19]:
tokenzied_dataset = data.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

## Model

In [16]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

In [18]:
model = AutoModelForTokenClassification.from_pretrained('bert-base-uncased', num_labels=9) #9 is the number of NER tags

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
model.to(device)

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, el

In [19]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
  output_dir = "./results",
  learning_rate=2e-5,
  evaluation_strategy="epoch",
  num_train_epochs = 3,
  per_device_train_batch_size=16,
  per_device_eval_batch_size=16,
  weight_decay=0.01,
)

data_collator = DataCollatorForTokenClassification(tokenizer)
metric = datasets.load_metric("seqeval", trust_remote_code=True)

<ipython-input-19-91da6bb84225>:14: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = datasets.load_metric("seqeval", trust_remote_code=True)


In [20]:
example = data['train'][0]
example

{'id': '0',
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}

In [21]:
label_list = data['train'].features['ner_tags'].feature.names
label_list

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [22]:
labels = [label_list[label_id] for label_id in example['ner_tags']]
labels

['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O']

In [23]:
metric.compute(predictions=[labels], references=[labels])

{'MISC': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2},
 'ORG': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

In [24]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)
    true_predictions = [[label_list[p] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]
    true_labels = [[label_list[l] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]
    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

## Training

In [25]:
trainer = Trainer(model=model, args=training_args, tokenizer=tokenizer,
                  data_collator=data_collator, train_dataset=tokenzied_dataset['train'],
                  eval_dataset=tokenzied_dataset['validation'], compute_metrics=compute_metrics)

trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.231400,0.062492,0.917942,0.926054,0.921980,0.982573
2,0.045900,0.056546,0.931386,0.941492,0.936412,0.985035
3,0.026400,0.057534,0.932716,0.945967,0.939295,0.985512


TrainOutput(global_step=2634, training_loss=0.07925995761187762, metrics={'train_runtime': 521.7231, 'train_samples_per_second': 80.738, 'train_steps_per_second': 5.049, 'total_flos': 1020143109346326.0, 'train_loss': 0.07925995761187762, 'epoch': 3.0})

In [32]:
model.save_pretrained("/content/drive/MyDrive/Deep Learning/Named Entity Recognition/ner_model")

## Testing

In [33]:
id2label = {
    str(i): label for i,label in enumerate(label_list)
}
label2id = {
    label: str(i) for i,label in enumerate(label_list)
}

In [36]:
import json

config = json.load(open("/content/drive/MyDrive/Deep Learning/Named Entity Recognition/ner_model/config.json"))

config['id2label'] = id2label
config['label2id'] = label2id

json.dump(config, open("/content/drive/MyDrive/Deep Learning/Named Entity Recognition/ner_model/config.json", "w"))

In [21]:
fine_tuned_model = AutoModelForTokenClassification.from_pretrained("/content/drive/MyDrive/Deep Learning/Named Entity Recognition/ner_model")

In [22]:
from transformers import pipeline

test_model = pipeline("ner", model=fine_tuned_model, tokenizer=tokenizer)

result = test_model("The CEO of Google, Sundar Pichai, visited the White House on Monday to discuss artificial intelligence with President Biden.")

print(result)

[{'entity': 'B-ORG', 'score': 0.98404044, 'index': 4, 'word': 'google', 'start': 11, 'end': 17}, {'entity': 'B-PER', 'score': 0.9985771, 'index': 6, 'word': 'sun', 'start': 19, 'end': 22}, {'entity': 'B-PER', 'score': 0.9988643, 'index': 7, 'word': '##dar', 'start': 22, 'end': 25}, {'entity': 'I-PER', 'score': 0.9987118, 'index': 8, 'word': 'pic', 'start': 26, 'end': 29}, {'entity': 'I-PER', 'score': 0.99860424, 'index': 9, 'word': '##hai', 'start': 29, 'end': 32}, {'entity': 'B-LOC', 'score': 0.9940953, 'index': 13, 'word': 'white', 'start': 46, 'end': 51}, {'entity': 'I-LOC', 'score': 0.9883357, 'index': 14, 'word': 'house', 'start': 52, 'end': 57}, {'entity': 'B-PER', 'score': 0.9982033, 'index': 23, 'word': 'bid', 'start': 118, 'end': 121}, {'entity': 'B-PER', 'score': 0.99872977, 'index': 24, 'word': '##en', 'start': 121, 'end': 123}]


In [23]:
for entity in result:
  print(f"Entity: {entity['word']:7} | Label:{entity['entity']}")

Entity: google  | Label:B-ORG
Entity: sun     | Label:B-PER
Entity: ##dar   | Label:B-PER
Entity: pic     | Label:I-PER
Entity: ##hai   | Label:I-PER
Entity: white   | Label:B-LOC
Entity: house   | Label:I-LOC
Entity: bid     | Label:B-PER
Entity: ##en    | Label:B-PER
